In [2]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
%matplotlib inline
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [6]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image
def draw_lines(img, lines, color=[0, 0,175], thickness=10):

    x_Nve,x_Pve,slope_Nve,slope_Pve= [],[],[],[]
    global x1_Pve
    global x2_Pve
    global x1_Nve
    global x2_Nve
    y_min = 0.6481482*img.shape[0]
    y_max = img.shape[0]
    count_Nve=0.
    count_Pve=0.
    sum_x1_Nve =0.
    sum_x2_Nve =0. 
    sum_x1_Pve =0.
    sum_x2_Pve =0.
    avg_x_Nve=0.

    for line in lines:
        for x1, y1, x2, y2 in line:
            if x2 != x1:
                m = (y2 - y1)/(x2 - x1)
                x_min=(y_min - y1)/m
                x_max=(y_max - y1)/m
                
                if -0.6 < m < -0.5:
                    count_Nve += 1
                    slope_Nve.append(m)
                    avg_slope_Nve=np.mean(slope_Nve)
                    x_Nve.append(x1)
                    x_Nve.append(x2)
                    avg_x_Nve=np.mean(x_Nve)
                    if abs(x1)<abs(avg_x_Nve)*0.9:
                        sum_x1_Nve += x_min + x1
                        sum_x2_Nve += x_max + x1
                    else:
                        sum_x1_Nve += x_min + avg_x_Nve
                        sum_x2_Nve += x_max + avg_x_Nve
                   
                if 0.55 > m > 0.45:
                    count_Pve += 1
                    slope_Pve.append(m)
                    avg_slope_Pve=np.mean(slope_Pve)
                    x_Pve.append(x1)
                    x_Pve.append(x2)
                    avg_x_Pve=np.mean(x_Pve)
                    if abs(x1)<abs(avg_x_Pve)*0.9:
                        sum_x1_Pve += x_min + x1
                        sum_x2_Pve += x_max + x1
                    else:
                        sum_x1_Pve += x_min + avg_x_Pve
                        sum_x2_Pve += x_max + avg_x_Pve

    if count_Nve != 0:
        x1_Nve = sum_x1_Nve/count_Nve
        x2_Nve = sum_x2_Nve/count_Nve
    cv2.line(img, (int(x1_Nve),int(y_min)), (int(x2_Nve),y_max), color, thickness)
    #cv2.fitLine(np.array([left_lines]), cv2.DIST_L2, 0, 0.01, 0.01)
    if count_Pve != 0:
        x1_Pve  = sum_x1_Pve/count_Pve
        x2_Pve  = sum_x2_Pve/count_Pve
    cv2.line(img, (int(x1_Pve),int(y_min)), (int(x2_Pve),y_max),  color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [4]:
def process_image(image):
    gray=grayscale(image)
    blur_gray =gaussian_blur(gray,3)
    edges=canny(blur_gray,75,150)
    masked_edges=region_of_interest(edges,np.array([[(150,540),(450,320),(490,310),(900,540)]]))
    h_img=hough_lines(masked_edges,2,np.pi/180,15,40,20)
    final=weighted_img(h_img,image)
    return final   

In [5]:
yellow_output = 'challenge_test.mp4'
clip2 = VideoFileClip('challenge.mp4')
yellow_clip = clip2.fl_image(process_image)
yellow_clip.write_videofile(yellow_output, audio=False)

NameError: name 'x1_Pve' is not defined

In [ ]:
lefts_x = np.array([])
lefts_y = np.array([])
rights_x = np.array([])
rights_y = np.array([])
for line in lefts:
    for x1,y1,x2,y2 in line:
        lefts_x = np.hstack((lefts_x, x1))
        lefts_x = np.hstack((lefts_x, x2))
        lefts_y = np.hstack((lefts_y, y1))
        lefts_y = np.hstack((lefts_y, y2))
    for line in rights:
        for x1,y1,x2,y2 in line:
            rights_x = np.hstack((rights_x, x1))
            rights_x = np.hstack((rights_x, x2))
            rights_y = np.hstack((rights_y, y1))
            rights_y = np.hstack((rights_y, y2))
left_coefs = np.polyfit(lefts_x, lefts_y, 1)
right_coefs = np.polyfit(lefts_x, lefts_y, 1)
left_y1 = y_size
left_x1 = int(round((left_y1 - left_coefs[1]) / left_coefs[0]))
left_y2 = upper_y
left_x2 = int(round((left_y2 - left_coefs[1]) / left_coefs[0]))
right_y1 = y_size
right_x1 = int(round((right_y1 - right_coefs[1]) / right_coefs[0]))
right_y2 = upper_y
right_x2 = int(round((right_y2 - right_coefs[1]) / right_coefs[0]))
rl_lines = [[[left_x1, left_y1, left_x2, left_y2],[right_x1, right_y1, right_x2, right_y2]]]
image_copy = np.copy(image)
image_lines = draw_lines(image_copy, rl_lines)
plt.figure()
plt.imshow(image_lines)

In [ ]:
# WHERE THE LANE LINES ARE IN THE IMAGE
left_bottom =[150,540]
right_bottom =[890,540]
apex = [470,315]
# Perform a linear fit (y=Ax+B) to each of the three sides of the triangle
# np.polyfit returns the coefficients [A, B] of the fit
fit_left = np.polyfit((left_bottom[0], apex[0]), (left_bottom[1], apex[1]), 1)
fit_right = np.polyfit((right_bottom[0], apex[0]), (right_bottom[1], apex[1]), 1)
fit_bottom = np.polyfit((left_bottom[0], right_bottom[0]), (left_bottom[1], right_bottom[1]), 1)


plt.imshow(lines_edges)
x = [left_bottom[0], right_bottom[0], apex[0], left_bottom[0]]
y = [left_bottom[1], right_bottom[1], apex[1], left_bottom[1]]
plt.plot(x, y, 'b', lw=2)

In [55]:
350/540

0.6481481481481481